In [25]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/drive.metadata.readonly"]


def main():
  """Shows basic usage of the Drive v3 API.
  Prints the names and ids of the first 10 files the user has access to.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("drive", "v3", credentials=creds)

    results = []
    folder_id = '1nZvAyc7Fl8UW7GTQDgTCWMN4rTFjXsfQ'

    def fetch_files(folder_id, folder_path):
    
      page_token = None
      while True:
          response = service.files().list(
              q=f"'{folder_id}' in parents",
              spaces='drive',
              fields='nextPageToken, files(id, name, mimeType)',
              pageToken=page_token
          ).execute()

          for file in response.get('files', []):
              if file['mimeType'] == 'application/vnd.google-apps.folder':
                  new_path = f"{folder_path}/{file['name']}"
                  fetch_files(file['id'], new_path)
              else:
                  results.append({
                      'file_name': file['name'],
                      'file_id': file['id'],
                      'folder_path': folder_path
                  })

          page_token = response.get('nextPageToken', None)
          if not page_token:
              break
  except HttpError as error:
      print(f'An error occurred: {error}')

  fetch_files(folder_id,"Root")
  return results

In [26]:
results = main()

In [36]:
# convert results to an dictionary where key is the folder path and value is a list of files
folder_files = {}
for result in results:
    if result['folder_path'] != "Root":
        folder_path = result['folder_path'].split("/")[-1]
        if folder_path not in folder_files:
            folder_files[folder_path] = []
        folder_files[folder_path].append(result['file_id'])

In [37]:
folder_files

{'nyc': ['1EZesQ4rA0vaGb4Hp_7f-8wtvJQiws_xd',
  '1sVU9BYyfuQMO1_0K-eKVNgK8ID6b2o8u',
  '17LezTzL0KBXalC_C4gX4N6PmjLUyfMAD',
  '19cTz5mPnSe75WttrIPWymIuxQxyVMv10',
  '1W0Ai5rAqBZYLy-zUDl1PvXDq3fTd27Ik',
  '13epn_-gtnPsc5uhoVKhagbLxce0iAkJu',
  '1h7gQzp836kdQnLBD95wH-C_2Wy2FwhoI',
  '1wWPqFKIKC9Mk884KhLf7mC1uYLMG52Y6',
  '1R2miTn3DupEY2KbPzl_JYv8yFzU30DUY',
  '1K4se1SANg7HBFkT6b14BKsBB0TBW6jav',
  '1j4RosVtlUEiflxfv5hDjb--gXOucSnZ0',
  '1rNolRCd54CA0XjtAIEiGm5VF9KRTeKAx',
  '1xQRFCc4ylZNHPv6fPCzlDaSdLMBrmoB6',
  '1hr2-YpmqX9hIDqfhOCe0N3jHxUZCMV-l',
  '10-XIlBDGu6-SJKJqA8gtdWf9cwvy-JQQ',
  '1KYiNM294njqegGLc_Yd-5BLtfD9D_FL_',
  '1-j4yHBhXTfDrNN5IgykLHywRH0jzj7Sn',
  '1CnFtfbN3DbiNHt4PYPv1dqACXRVAbWcw',
  '1m1BBaZLeWY3A8KSJTD72E9VQcNPdk7_O',
  '1iVyxanLrBlRkfxXzfbi04ncZTE0a0W1I',
  '1ez_VPgIK1-WmKRjd0YdxyH7ZwEXahQ6W',
  '1Kw6oL2oCgztjtz1RObWjsaHdLQI9zbqr'],
 'stk': ['1PRi_elXLjEux7RO9fy5IACbp6Jtqz0El',
  '1O_m-nLA-hi4Mw6L669jkJPyi7NmdS6eM',
  '1QyaExUIpz8PrDrtOCOkfPJZIEIIuCWpn',
  '1ryzbPK

In [28]:
#count numner of itesm for each folder_path
folder_path_count = {}
for result in results:
  folder_path = result['folder_path']
  if folder_path in folder_path_count:
    folder_path_count[folder_path] += 1
  else:
    folder_path_count[folder_path] = 1

In [29]:
folder_path_count

{'Root/nyc': 22,
 'Root/stk': 9,
 'Root/cali': 11,
 'Root/other': 28,
 'Root/chi': 7,
 'Root': 5}